# Mic2img

**Notes**


If you get a 403 Forbidden message when trying to open the server window, please try copying the notebook to your own drive.

This notebook requires high ram. Ideally we a A100 or V100 Other GPUs may lead to issues.

---

If you run into any trouble or have suggestions, please create an issue on my [github](https://github.com/scyheidekamp) or message me on [Discord](https://discord.gg/yxwgJPzZ)

In [ ]:
#@title Check GPU status
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-6d2efa25-99c3-8d3e-0afc-ed044dea4363)


In [ ]:
#@title Mount Google Drive
from pathlib import Path
import os

drive_mounted = False
gdrive_fpath = '.'
local_path = '/content/'



#@markdown Mounting your google drive is optional.
#@markdown If you mount your drive, the text and image files will be stored on it.

mount_gdrive = True # @param{type:"boolean"}
if mount_gdrive and not drive_mounted:
    from google.colab import drive

    gdrive_mountpoint = '/content/drive/' #@param{type:"string"}
    gdrive_subdirectory = 'MyDrive/mic2emo2img' #@param{type:"string"}
    gdrive_fpath = str(Path(gdrive_mountpoint) / gdrive_subdirectory)
    try:
        drive.mount(gdrive_mountpoint, force_remount = True)
        !mkdir -p {gdrive_fpath}
        %cd {gdrive_fpath}
        local_path = gdrive_fpath
        drive_mounted = True
    except OSError:
        print(
            "If you received an OSError and your drive"
            " was already mounted, ignore it."
            )
        raise

Mounted at /content/drive/
/content/drive/MyDrive/mic2emo2img


In [ ]:
#@title Install dependencies
!pip install ffmpeg
!pip install torch
!pip install diffusers==0.8.0 transformers ftfy
!pip install accelerate

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=6d0f55fdee7b971e6eb9d36bcd8d83f0c04c0d4284f276c96d99248ebba7c55b
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 22.7 MB/s eta 0:00:00


In [ ]:
#@title Install Whisper speech recognition
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-jpj2_viq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-jpj2_viq
  Resolved https://github.com/openai/whisper.git to commit b91c907694f96a3fb9da03d4bbdc83fbcd3a40a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798285 sha256=c9e3dc3e8e03615025c9dde996fdb710232fbc808273fe2ddc004aaa81a5deb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-0sd715rp/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


#Install Stable Diffusion

Choose a stable diffusion model by taking a huggingface model-card link and removing everything except for the name/model

i.e. huggingface.co/stabilityai/stable-diffusion-2-1 becomes stabilityai/stable-diffusion-2-1.

This is what you need to add to the "model_card"

Check out https://huggingface.co/sd-dreambooth-library for custom Stable Diffusion models.

In [ ]:
import requests
import torch
from PIL import Image
from io import BytesIO

from diffusers import StableDiffusionImg2ImgPipeline
device = "cuda"
#model_card = "stabilityai/stable-diffusion-2-1"#@param{type:"string"}

model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)

Fetching 27 files:   0%|          | 0/27 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [ ]:
#@title Install keywords and emotions
!pip install keybert
!pip install transformers -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23777 sha256=cfe0996ae7893655a8c198ed92db44949c1181404960e6e20aa8375902cc7c53
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=f510988194dad0c97141a0c7f6e35919c9c1aa1f7d46f4a9083bd0bab73f4ea6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built keybert sentence-transformers


In [ ]:
from keybert import KeyBERT
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
kw_model = KeyBERT()
tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
emotion = pipeline('text-classification', model='arpanghoshal/EmoRoBERTa', return_all_scores=True)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_a

In [ ]:
#@title Global settings

#@markdown Width and height should be divisable by 8
width = 768 #@param {type:"integer"}
height = 512 #@param {type:"integer"}

prompt_modifier = "abstract image, high quality, 8K, UHD"#@param {type:"string"}
negative_prompt= "text, people"#@param {type:"string"}

directory_name = "workshop_actual5"#@param {type:"string"}

amount_of_emotions = 4 #@param {type:"slider", min:0, max:10, step:1}

init_path = "/content/drive/MyDrive/mic2emo2img/shape4.png" #@param {type:"string"}

In [ ]:
#@title Server related stuff
hostName = "0.0.0.0"
serverPort = 8000

output_server = True #@param {type:"boolean"}

# Python 3 server example
from http.server import BaseHTTPRequestHandler, HTTPServer
from google.colab.output import eval_js
import time
import base64

class MyServer(BaseHTTPRequestHandler):
  def do_GET(self):
    fbase64_string = None
    if os.path.exists(f"{gdrive_fpath}/{directory_name}/latest.png"):
      f = open(f"{gdrive_fpath}/{directory_name}/latest.png", "rb")
      fbase64_string = base64.b64encode(f.read()).decode('ascii')
    self.send_response(200)
    self.send_header("Content-type", "text/html")
    self.end_headers()
    self.wfile.write(bytes("<html style=\"background-color:#000000;text-align:center;\"><head><title>Latest image</title></head>", "utf-8"))
    self.wfile.write(bytes("<body>", "utf-8"))
    if fbase64_string is not None:
      self.wfile.write(bytes("<img style=\"width:auto;height:100%;max-height:vh;\" src=\"data:image/jpeg;base64," + fbase64_string + "\"/>", "utf-8"))
    self.wfile.write(bytes("<script> setTimeout(() => { location.reload() }, 2000) </script>", "utf-8"))
    self.wfile.write(bytes("</body></html>", "utf-8"))

  def log_message(self, format, *args):
    return

if output_server == True:
  print(eval_js("google.colab.kernel.proxyPort(" + str(serverPort) + ")"))
else:
  print("Server inactive")

https://vy1gojnuz3l-496ff2e9c6d22116-8000-colab.googleusercontent.com/


Click this link above to display the images in a new window

Refresh the page after starting the loop below, it should start displaying images within a minute

# Start the loop



In [ ]:
# imports
import ipywidgets as widgets
from threading import Thread
from queue import Queue
import requests
from io import BytesIO
from IPython.display import Javascript, display, Image
from google.colab import output
from base64 import b64decode
import time
import subprocess
import whisper
import os
import signal
import sys
from PIL import Image
import os
import multiprocessing
from torch import autocast

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  return b

messages = Queue()
recordings = Queue()

textblock = widgets.Output()

def speech_recognition(textblock):
  counter = int(0)
  while not messages.empty():
    frames = recordings.get()

    if len(frames) > 0:

      if os.path.exists("/content/audio.webm"):
        os.remove("/content/audio.webm")
      if os.path.exists("/content/audio.wav"):
        os.remove("/content/audio.wav")

      with open('/content/audio.webm','wb') as f:
        f.write(frames)
        command = ['ffmpeg', '-i', "/content/audio.webm", "/content/audio.wav"]
        subprocess.run(command,stdout=subprocess.PIPE,stdin=subprocess.PIPE)

        model = whisper.load_model("large")


        audio = whisper.load_audio("/content/audio.wav")
        audio = whisper.pad_or_trim(audio)

        mel = whisper.log_mel_spectrogram(audio).to(model.device)
        _, probs = model.detect_language(mel)
        language = max(probs, key=probs.get)
        print(f"Detected language: {max(probs, key=probs.get)}")
        options = whisper.DecodingOptions(fp16=False)
        result = whisper.decode(model, mel, options)
        if language != "en":
          translate = model.transcribe(audio, task = 'translate')
          print("translated: ")
          work_text = translate["text"]
          print(work_text)
        else:
          work_text = result.text
          print(work_text)

        sentence = Prompt(work_text)
        keywords = sentence.extract_keywords()
        emo = sentence.extract_emotions()
        final_prompt = keywords + emo + prompt_modifier
        print(final_prompt)

        if counter < 1:
          init_img = init_path
        else:
          imgcounter = counter-1
          init_img = f"{directory_name}/{imgcounter:04d}.png"

        init_images = Image.open(init_img).convert("RGB").resize((768,768))
        generator = torch.Generator(device=device).manual_seed(1024)
        image = pipe(prompt=final_prompt, negative_prompt=negative_prompt, init_image=init_images, strength=0.55, guidance_scale=7.5).images
        pipe.safety_checker = lambda images, clip_input: (images, False)

        if not os.path.exists(directory_name):
          os.mkdir(directory_name)
        display(image)
        image[0].save(f"{directory_name}/latest.png")
        img_path = f"{directory_name}/{counter:04d}.png"
        image[0].save(img_path)


        with open(f"{directory_name}/recorded.txt", 'a') as f:
          f.write("\n" + work_text)

        counter += 1
        frames = []

class Prompt:

  def __init__(self, sentence):
    self.sentence = sentence

  def extract_keywords(self):
    keyword_list = kw_model.extract_keywords(self.sentence, keyphrase_ngram_range=(1, 3), use_mmr=True, diversity=0.9)
    result = [x[0] for x in keyword_list]
    return str(result)

  def extract_emotions(self):
    emotion_labels, = emotion(self.sentence)
    sorted_emotions = sorted(emotion_labels, key=lambda item: item['score'], reverse=True)
    emo_result = []
    for i in sorted_emotions[:amount_of_emotions]:
      val = list(i.values())
      text = (val[0])
      emo_result.append(text)
    return str(emo_result)

def start_recording():
  print(' ')
  print('Start recording')
  messages.put(True)

  transcribe = Thread(target=speech_recognition, args=(textblock,))
  transcribe.start()

  while True:
    frames = record(30)
    print('Finished recording')
    recordings.put(frames)
    time.sleep(0.1)

def start_server():
  webServer = HTTPServer((hostName, serverPort), MyServer)
  if output_server == True:
    print("Server started http://%s:%s" % (hostName, serverPort))

  try:
      webServer.serve_forever()
  except:
      pass

  webServer.server_close()
  print("Server stopped.")

if output_server == True:
  server = multiprocessing.Process(target=start_server)
  server.start()

def signal_handler(sig, frame):
  server.terminate()  # sends a SIGTERM
  sys.exit(0)


signal.signal(signal.SIGINT, signal_handler)

start_recording()
display(textblock)

Server started http://0.0.0.0:8000 
Start recording


<IPython.core.display.Javascript object>


Finished recording


<IPython.core.display.Javascript object>

100%|██████████████████████████████████████| 2.87G/2.87G [00:27<00:00, 111MiB/s]


...

Detected language: en
so well. This is Sky. And what do we do with the blog? Do we put it somewhere here? We should take it upstairs. Yeah? Are you going to leave your bag here? Are you going to go home and then back? Are you? No. Because the recording is on. Say. Speaking, because it's just a few of my songs. Yeah. So I don't play them.
['sky blog upstairs', 'recording say speaking', 'yeah', 'bag going', 'don play']['neutral', 'disapproval', 'curiosity', 'confusion']abstract image, high quality, 8K, UHD


  0%|          | 0/28 [00:00<?, ?it/s]

Finished recording


<IPython.core.display.Javascript object>

[<PIL.Image.Image image mode=RGB size=768x768 at 0x7F03A8931240>]